# Actividad 10 - Mecanismos de Votación
### Desafío 1 - Preparación del Ambiente de Trabajo
- Describa el comportamiento de las variables.
- Dada la naturaleza de los atributos, es probable que algunas mediciones estén correlacionadas entre sí. Para ello, genere un diagnóstico previo de multicolinealidad utilizando la función identify_high_correlations que se encuentra en el archivo helpers.py . Para todos aquellos atributos que tengan una correlación de .8, reporte sus nombres.
- Antes de generar los conjuntos de entrenamiento y validación, preprocese los datos con los siguientes pasos:
- Recodifique la variable shares en una variable binaria que identifique como 1 todos los registros con más de 1400 "compartir" y 0 de lo contrario. Para evitar multicolinealidad, elimine la variable shares posteriormente.
- Elimine todas las variables que presentaban una correlación mayor a .8. Este paso es para evitar la multicolinealidad de los atributos.
- Genere un análisis de Componentes Principales para extraer las principales 30 dimensiones. Guarde estas dimensiones en un nuevo objeto.

In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import helpers
import re
seed = 602

In [236]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import VotingClassifier

In [237]:
df = pd.read_csv('./OnlineNewsPopularity/OnlineNewsPopularity.csv')

In [ ]:
# las etiquetas de las columnas presentan un espacio extra, con ésto lo podemos eliminar
df.columns = [i.replace(' ', '') for i in df.columns]
# eliminamos el string de url que no sirve para el análisis
df = df.loc[:, 'n_tokens_title':'shares']
# generamos el conjunto de variables
qnty = df.filter(regex='^n_', axis=1)
channel = df.filter(regex='^data_', axis=1)
days = df.filter(regex=re.compile("weekday|weekend"), axis=1)
sentiments = df.filter(regex=re.compile("negative|positive|subjectivity"), axis=1) 
lda = df.filter(regex='^LDA_\d', axis=1)
keywords = df.filter(regex='^kw_', axis=1)

In [ ]:
plt.figure(figsize=(12,25))
cols = 5
rows = np.ceil(df.shape[1] / cols)

for i, (colname, serie) in enumerate(df.iteritems()):
    plt.subplot(rows,cols,i+1)
    if (len(serie.value_counts()) > 2):
        sns.distplot(serie)
    else:
        sns.countplot(serie)
        
plt.tight_layout(pad=1)

 - Tenemos un número de atributos binarios, especialmente respecto al día y la categoría del artículo
 - Algunas variables tienen comportamiento relativamente normal como `global_subjectivity`, `global_sentiment_polarity`, `global_rate_positive_words`, `avg_positive_polarity`.
 - Luego otro grupo de atributos son evaluaiones de 0 a 1/-1, que corresponden a tasas

In [ ]:
#Mostramos los valores de correlaciones mayores a 0.8. Aprovechamos de arreglar el helpers ajajaj

corr = helpers.identify_high_correlations(df,threshold = 0.8)
print(corr)

In [ ]:
df = df.drop(columns=corr['var2'])

In [ ]:
df['shares'] = np.where(df['shares'] > 1400, 1, 0)

In [ ]:
df['shares'].value_counts()

In [ ]:
pca = PCA(n_components=30)
pca_dim = pca.fit_transform(df.drop(columns=['shares']))

### Desafío 2 - Evaluación de modelos individuales
- A continuación generará una serie de modelos que se incorporarán posteriormente al comité de votación. Para ello, se solicita que:
  - Importe los módulos correctamente.
  - Para cada uno de ellos, genere un reporte en las métricas de desempeño respecto a
  - Precision , Recall , F1 . Puede hacer uso de la función plot_classification_report disponible en el archivo helpers.py .
  - Comente el desempeño general de cada uno

- La lista de modelos es la siguiente. Cabe destacar que la mayoría de éstos corresponden a implementaciones vanilla, salvo que se indique lo contrario:
  - Regresión Logística.
  - Algoritmo de KMedias.
  - Árbol de Clasificación con un max_depth=1 .
  - Árbol de Clasificación con un max_depth=4 .

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(pca_dim, df['shares'], test_size=.33, random_state=seed)

In [ ]:
ensamble = [
    ('Logistic Regression', LogisticRegression(random_state=seed)),
    ('Muñon Tree', DecisionTreeClassifier(max_depth=1, random_state=seed)),
    ('Four Tree', DecisionTreeClassifier(max_depth=4, random_state=seed)),
    ('Kmeans', KMeans(n_clusters=2, random_state=seed))
]

In [ ]:
plt.figure(figsize=(10,10))
cr =  dict()
for index, i in enumerate(ensamble):
    yhat = i[1].fit(X_train, y_train).predict(X_test)
    plt.subplot(2,2,index+1)
    plt.title(i[0])
    helpers.plot_classification_report(y_test, yhat)
    if (index  == 1):
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xlim((0, 1.0))
    cr.update({i[0]: pd.DataFrame(classification_report(y_test, yhat, output_dict=True))})

- El Modelo Kmeans presenta los peores resultados para identificar la Clase 1, pero los mejores para la clase 0
- El árbol de profundidad 4 y la regresión logística presentan los mejores resultados. El primero presenta resultados mejores para la clase 1 y el segundo para la clase 0.

### Desafío 3- Entrenamiento de Comité
- Entrene el comité de clasificadores sin modificar el esquema de votación.
- Reporte el desempeño a nivel de cada clase para cada métrica.

In [ ]:
voting_classifier = VotingClassifier(ensamble).fit(X_train, y_train)
yhat_vc = voting_classifier.predict(X_test)

plt.figure()
plt.title('Clasificación por Votación')
helpers.plot_classification_report(y_test, yhat_vc)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlim((0, 1.0));

cr.update({'Voting': pd.DataFrame(classification_report(y_test, yhat_vc, output_dict=True))})


In [ ]:
aux = pd.DataFrame()
for i in cr:
    aux[i]=cr[i]['0']
aux
    

In [ ]:
aux = pd.DataFrame()
for i in cr:
    aux[i]=cr[i]['1']
aux

- Para la Clase 0, el mejor modelo es el de Voting considerando la métrica f1. Sin embargo la mejor presicion correponde al árbol de decisión de 4 de profundidad. KMeans de los que predijo como positivos, un buen porcentaje (92%) fue correcto
- Para la Clase 1, el mejor modelo es Four Tree. Respecto la presición de los datos correctos, el Voting tuvo mejor resultado. En este caso KMeans tiene un muy mal porcentaje de recall.
- Podemos decir que KMeans sobreestima las clases 0, lo que explica su alto recall para esa clase. Efecto similar ocurre con el muñon de decisión. 



### Desafío 4 - Calibración de Comité con Ponderadores
- El base al comportamiento de los clasificadores individuales del ensamble, proponga dos esquemas de ponderación para mejorar el desempeño del modelo.
- Reporte el desempeño del mejor ensamble heterogéneo.

In [ ]:
weights_hyperparams = {
'AumentoLR/FT': [.375, .125, .375, .125],
'SoloLR/FT': [.5, .0, .5, .0],
}

In [ ]:
voting_classifier2 = VotingClassifier(ensamble, weights=weights_hyperparams['AumentoLR/FT']).fit(X_train, y_train)
yhat_vc = voting_classifier2.predict(X_test)

plt.figure()
plt.title('Clasificación por Votación AumentoLR/FT')
helpers.plot_classification_report(y_test, yhat_vc)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlim((0, 1.0));

cr.update({'Voting AumentoLR/FT': pd.DataFrame(classification_report(y_test, yhat_vc, output_dict=True))})

In [ ]:
voting_classifier3 = VotingClassifier(ensamble, weights=weights_hyperparams['SoloLR/FT']).fit(X_train, y_train)
yhat_vc = voting_classifier3.predict(X_test)

plt.figure()
plt.title('Clasificación por Votación SoloLR/FT')
helpers.plot_classification_report(y_test, yhat_vc)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlim((0, 1.0));

cr.update({'Voting SoloLR/FT': pd.DataFrame(classification_report(y_test, yhat_vc, output_dict=True))})

In [ ]:
aux = pd.DataFrame()
for i in cr:
    aux[i]=cr[i]['0']
aux

In [ ]:
aux = pd.DataFrame()
for i in cr:
    aux[i]=cr[i]['1']
aux

- El mejor ensamble es el que solo considera la Regresión Logística y el Árbol de Decisión de 4 de profundidad para aquellas publicaciones con menos _shares_
- El mismo árbol es el mejor modelo para predecir publicaciones populares, pero de los ensambles el que pondera de menor manera los modelos KMeans y muñón de decisión son los mejores. 
- Sin embargo considerando precision el modelo SoloLR/FT es aquel que logra predecir aquellos publicaciones que verdaderamente son populares, pero entrega muchos falsos negativos. 